[View in Colaboratory](https://colab.research.google.com/github/rdorff/DeepLearning/blob/master/Lab6.ipynb)

In [0]:
! wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
! tar -xzf text_files.tar.gz
! pip install unidecode
! pip install torch

In [3]:
import unidecode
import string
import random
import re
 
import pdb
 
all_characters = string.printable
n_characters = len(all_characters)
 
file = unidecode.unidecode(open('./text_files/lotr.txt').read())
file_len = len(file)
print('file_len =', file_len)

file_len = 2579888


In [4]:
chunk_len = 200
 
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]
 
print(random_chunk())

e); and the guests were selected from all the families to which 
Bilbo and Frodo were related, with the addition of a few special unrelated 
friends (such as Gandalf). Many young hobbits were included,


In [5]:
import torch
from torch.autograd import Variable
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(tensor)
 
print(char_tensor('at its foot three Elves were'))

tensor([10, 29, 94, 18, 29, 28, 94, 15, 24, 24, 29, 94, 29, 17, 27, 14, 14, 94,
        40, 21, 31, 14, 28, 94, 32, 14, 27, 14])


In [0]:
def random_training_set():    
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
 
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
 
 
        # encode using embedding layer
        self.embedding = nn.Embedding(input_size, hidden_size)
        # set up GRU passing in number of layers parameter (nn.GRU)
        self.gru = GRU(input_size, hidden_size, n_layers)
        
        # decode output   
        
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
 
    def forward(self, input_char, hidden):
        # by reviewing the documentation, construct a forward function that properly uses the output
        # of the GRU
        
        output = self.embedding(input_char).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        
        return output, hidden
        

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [0]:
class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, n_layers=1):
    super(GRU, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.n_layers = n_layers
    self.x2h = nn.Linear(input_size, 3*hidden_size)
    self.h2h = nn.Linear(hidden_size, 3*hidden_size)


  def forward(self, x, hidden):
    x = x.view(x.size(1),-1)
   
    gate_x = self.x2h(x) .squeeze()
    gate_h = self.h2h(hidden).squeeze()
    
    i_r, i_i, i_n = gate_x.chunk(3, 0)
    h_r, h_i, h_n = gate_h.chunk(3, 0)
        
        
    reset_gate = torch.sigmoid(i_r + h_r)
    update_gate = torch.sigmoid(i_i + h_i)
    new_gate = torch.tanh(i_n + (reset_gate * h_n))

    h_t = new_gate - new_gate * update_gate + update_gate * hidden 
    
    return h_t, h_t

In [0]:
def train(inp, target, decoder, decoder_optimzier, criterion):
    ## initialize hidden layers, set up gradient and loss 
      # your code here
    ## /
    hidden = decoder.init_hidden()
    decoder_optimizer.zero_grad()
    loss = 0
    
    for c in range(chunk_len):
        output, hidden = decoder(inp[c], hidden) # run the forward pass of your rnn with proper input
        loss += criterion(output, target[c].unsqueeze(0))   
   
    
    ## calculate backwards loss and step the optimizer (globally)
      # your code here
    ## /
 
    loss.backward()
    decoder_optimizer.step()
  
  
    return loss.item() / chunk_len

In [0]:
def evaluate(decoder, prime_str='A', predict_len=100, temperature=0.8):
    ## initialize hidden variable, initialize other useful variables 
      # your code here
    ## /
    hidden = decoder.init_hidden()
    predicted = prime_str
    
      
    prime_input = char_tensor(prime_str)
 
    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
        
    inp = prime_input[-1]
 
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden) #run your RNN/decoder forward on the input
      
 
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
 
        ## get character from your list of all characters, add it to your output str sequence, set input
        ## for the next pass through the model
         # your code here
        ## /
        
        predicted = predicted + all_characters[top_i]
 
    return predicted

In [70]:
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch import optim
import numpy as np
import time


n_epochs = 2000
print_every = 100
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.005
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0
 
for epoch in range(1, n_epochs + 1):
    loss_ = train(*random_training_set(), decoder, decoder_optimizer, criterion)       
    loss_avg += loss_
    
 
    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
        print(evaluate(decoder, 'Wh', 100), '\n')
 
    if epoch % plot_every == 0:
        all_losses.append(loss_avg / plot_every)
        loss_avg = 0

[18.380439519882202 (100 5%) 2.4508]
Wheaeeee,eepeeeaee ueeeeeei,ieeeaeaeaeieateee eeetee ee eeue eeee.;eeeeeieeeeeaeeeee e eeee eeeeeeeo,e 

[36.80793833732605 (200 10%) 2.0299]
Whteieetae eeieeeet eeeettee aeaeeete i eiteie  e  ieetea e ae ie t' e  eai  e ee  et ee ioeeeeaieaiee 

[55.307148933410645 (300 15%) 2.1307]
Whe  iei   ii  t i  ye ia .tee  ii  itieaei  taa ia eae ti i ttaea i a  ia ee  oiiiiit  o  iiea a aaia 

[73.83614993095398 (400 20%) 1.8632]
Wheee   aa ie e    iei e eeee e eee.i i e oeety     i e i    ei os  i  , tei     t i t  t  ei ii a eee 

[92.35108065605164 (500 25%) 2.1072]
Whe eeiaoeaaaiooii   aieoataeoiiea i aeeaoii iei oe meeieeoaeeaeie   e ee  eoeaeii a i teia .oeait iee 

[110.89212036132812 (600 30%) 1.8382]
Who iieeieie ye etat  t eea  eeeieeeiei heto ea  tiii t iae e  s eaiaeie iieeeeieee  ietea ae t  it  t 

[129.3649935722351 (700 35%) 1.9604]
Whee t t    e  e ee    ie    at  et ei e  e    t e s e. aa     i  eie i   i , ae  ei  eee te e. et i   

[147